In [31]:
import discogs_client

In [32]:
def readUserToken():
    try:
        with open("user_token.env", "r") as file:
            content = file.read()
            return content
    except FileNotFoundError:
        return {}
        
d = discogs_client.Client('dardok_app/0.1', user_token='{}'.format(readUserToken().rstrip()))

In [33]:
artist_id = 1451990
min_average = 4.8

In [34]:
print(dir(d))

result_artist = d.artist(artist_id)

print()
print(dir(result_artist))

print()
print(dir(result_artist.releases))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_access_token_url', '_authorize_url', '_base_url', '_check_user_agent', '_delete', '_fetcher', '_get', '_patch', '_post', '_put', '_request', '_request_token_url', 'artist', 'backoff_enabled', 'connection_timeout', 'fee_for', 'get_access_token', 'get_authorize_url', 'identity', 'label', 'list', 'listing', 'master', 'order', 'read_timeout', 'release', 'search', 'set_consumer_key', 'set_timeout', 'set_token', 'user', 'user_agent', 'verbose']

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__

In [37]:
# COMMON FUNCTIONS

def getFilteredReleaseListFromArtist(artist):
    """ A function that returns all the filtered releases from artist """
    filtered_release_list = []
    print()
    print("Processing artist: {} {}...".format(artist.name, artist.url))
    for k in range(1, result_artist.releases.pages+1):
        print()
        print("Page: {}/{}".format(k,result_artist.releases.pages))
        print(result_artist.releases.page(k))
        for item in result_artist.releases.page(k):
            release = d.release(item.id)       
            if release.community.rating.average >= min_average and release.formats[0]['name']=='Vinyl' and 'Electronic' in release.genres:
                filtered_release_list.append(release)
    print(filtered_release_list)
    # return filtered_release_list


In [38]:
getFilteredReleaseListFromArtist(result_artist)


Processing artist: Asusu https://www.discogs.com/artist/1451990-Asusu...

Page: 1/1
[<Release 2014207 'Small Hours / Taurean'>, <Release 2296396 'Togetherness / Horizons (Asusu Remix)'>, <Release 2498307 'No Kya'>, <Master 413449 'Sister'>, <Release 4663619 'Velez / Rendering'>, <Release 5013363 'Surge / Remnants'>, <Release 5763421 'MM/KM & Dresvn Remixes'>, <Master 745601 'A Made Up Sound Remixes 1 & 2'>, <Master 747214 'MMM & Pangaea Remixes'>, <Master 744015 'Surgeon & Nick Höppner Remixes'>, <Release 6661530 'Serra EP'>, <Release 7631446 'Secret Thirteen Mix 156'>, <Release 7747820 'Trephine'>, <Release 9155008 'Hallucinator '>, <Master 823811 'Decay Versions Pt. 1'>, <Master 1694579 'Remixed'>, <Release 10447414 'Remixes - 1'>, <Release 6009879 'Broadwalk And Friends'>, <Master 233632 'DubKraft Records - The First 3 Years'>, <Release 5586682 'A Colourful Storm 002'>, <Release 17882686 'Dub is Eternal - The 420 Dubkraft Anthology'>, <Release 5644401 'Podcast 322'>, <Release 77341

HTTPError: 404: Release not found.